In [1]:
from langchain_core.prompts import PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import ArxivLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 加载arXiv上的论文ReAct: Synergizing Reasoning and Acting in Language Models
loader = ArxivLoader(query = "2210.03629", load_max_docs = 1)
docs = loader.load()
print(docs[0].metadata)

# 把文本分割成500个字符为一组的片段
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
chunks = text_splitter.split_documents(docs)

# 构建Stuff形态（即文本直接拼合）的总结链
doc_prompt = PromptTemplate.from_template("{page_content}")
chain = (
    {
        "content": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | PromptTemplate.from_template("使用中文总结以下内容，不需要人物介绍，字数控制在300个字符以内：\n\n {content}")
    | ChatOllama(model = "qwen2.5")
    | StrOutputParser()
)

# 由于论文很长，所以我们只选取前2000个字符作为输入并调用总结链
chain.invoke(chunks[:4])

{'Published': '2023-03-10', 'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models', 'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao', 'Summary': 'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g.\naction plan generation) have primarily been studied as separate topics. In this\npaper, we explore the use of LLMs to generate both reasoning traces and\ntask-specific actions in an interleaved manner, allowing for greater synergy\nbetween the two: reasoning traces help the model induce, track, and update\naction plans as well as handle exceptions, while actions allow it to interface\nwith external sources, such as knowledge bases or environments, to gather\nadditional information. We apply our approach, named ReAct, to a diverse set of\nlanguage an

'该研究发表于2023年ICLR会议，题为“REACT：语言模型中推理与行动的协同”。研究探索了大型语言模型（LLMs）在生成推理过程和任务特定行动方面的综合能力。通过交互方式，推理帮助模型生成、跟踪和更新行动计划，并处理异常情况；而行动则允许模型从外部知识库或环境中获取额外信息。实验表明，ReAct 在问答与事实验证等任务上优于基准方法，在人类可解释性和可信度方面也表现更佳。在两个互动决策基准测试中，ReAct 的成功率分别比模仿学习和强化学习高出了34% 和10%，同时仅使用简单的维基百科API即解决了链式推理中的幻觉和错误传播问题。'